# Title + Exec Sum.

## Config & Utils

## Data Ingest

## Pair Selection

## Hedge Ratio & Spread

## OU Check

## Signals & Sizing

## Cost and Execution

## Walk-Forard Backtest

## Results

## Factor Neautrality

## Sensitivity Sweeps

## Regime Splits

## OOS Holdout

## Beta Stability

## Structural Breaks

## Cost Stress Test

## ML Ranker for Pairs

## Intraday Extensions